## Noisy Quantum Computer Simulator Demo

First, include the quantum computer simulator:

In [1]:
using NoisyQuantumComputerSimulator

### A simple 1 qubit gate

Let's start with creating a circuit for a single qubit register and use `+=` to append gates to the circuit. Run the circuit by calling `exec()` and get the resulting density matrix:

In [2]:
c = Circuit(1)
c += X(0)
c += H(0)
exec(c)

2×2 Array{Complex{Float64},2}:
  0.5+0.0im  -0.5+0.0im
 -0.5+0.0im   0.5+0.0im

### A simple 2 qubit gate
Now, let's create a 2 qubits circuit and apply Hadamard to qubit 1 and then CNOT to the two qubits, where qubit 1 is control and qubit 0 is target. This time we'll specify all gates in the circuit constructor (as an opposite to using `+=`). The result is the density matrix for the Bell state:

In [3]:
exec(Circuit(2, H(1), CNOT(1, 0)))

4×4 Array{Complex{Float64},2}:
 0.5+0.0im  0.0+0.0im  0.0+0.0im  0.5+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.5+0.0im  0.0+0.0im  0.0+0.0im  0.5+0.0im

### Parametric circuits

Now, let's create a parametric circuit. First of all, we can rotate around X axis by a constant angle like this:

In [4]:
exec(Circuit(1, RX(Float64(pi/2.0), 0)))

2×2 Array{Complex{Float64},2}:
 0.5+0.0im  0.0+0.5im
 0.0-0.5im  0.5+0.0im

`RX()` can also accept a parameter name (as an opposite to a constant angle value) which value will be specified when calling `exec()`. In this case we create a parametric circuit which we can run several times with different values of the parameter(s):

In [5]:
c = Circuit(1, RX("theta", 0))
exec(c, Dict("theta" => Float64(pi)))

2×2 Array{Complex{Float64},2}:
 3.7494e-33+0.0im          0.0+6.12323e-17im
        0.0-6.12323e-17im  1.0+0.0im        

Reset state back to initial 0 state before running the circuit with a different parameter:

In [6]:
reset_state!(c)
exec(c, Dict("theta" => Float64(pi / 2)))

2×2 Array{Complex{Float64},2}:
 0.5+0.0im  0.0+0.5im
 0.0-0.5im  0.5+0.0im

### Custom gates
A custom gate can also be specified:

In [7]:
some_matrix = Matrix{Complex{Float64}}([0 1; 1 0])
exec(Circuit(1, Gate(some_matrix, 0)))

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

### Parametric custom gates
To specify custom parametric gate use a function that returns custom matrix that depends on dictionary value(s) passed to this function by `exec()`:

In [8]:
matrix_func = function (params)
    angle = params["alpha"]
    return Matrix{Complex{Float64}}([cos(angle / 2) complex(0, -1) * sin(angle / 2); complex(0, -1) * sin(angle / 2) cos(angle / 2)])
end
c = Circuit(1, Gate(matrix_func, 0))
exec(c, Dict("alpha" => Float64(pi)))

2×2 Array{Complex{Float64},2}:
 3.7494e-33+0.0im          0.0+6.12323e-17im
        0.0-6.12323e-17im  1.0+0.0im        

Reset state back to initial 0 state before running the circuit with a different parameter value:

In [9]:
reset_state!(c)
exec(c, Dict("alpha" => Float64(pi / 2)))

2×2 Array{Complex{Float64},2}:
 0.5+0.0im  0.0+0.5im
 0.0-0.5im  0.5+0.0im

### Adding control bits to a gate
You can add one or several control qubits to any gate:

In [10]:
exec(Circuit(2, controlled(1, X(0))))

4×4 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [11]:
exec(Circuit(3, controlled([0,2], X(1))))

8×8 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im

### Adding noise
Let's add amplitude dumping to `X` gate applied to qubit 0 with probability 0.5:

In [12]:
exec(Circuit(1, damp_amplitude(X(0), 0.5)))

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

### Adding custom noise
A custom noise can be added to gates via Kraus operators. Let's define `damping_residual_kraus` that represents no noise, `damping_kraus` that represents decay ket 1 to ket 0 with given probability and `damping_kraus_map` that combines the two Kraus operators:

In [13]:
damping_residual_kraus(decay_1_to_0_probability) = Matrix{Complex{Float64}}([1 0; 0 √(1 - decay_1_to_0_probability)])
damping_kraus(decay_1_to_0_probability) = Matrix{Complex{Float64}}([0 √decay_1_to_0_probability; 0 0])
damping_kraus_map(decay_1_to_0_probability) = [damping_residual_kraus(decay_1_to_0_probability), damping_kraus(decay_1_to_0_probability)]
exec(Circuit(1, noisify(X(0), damping_kraus_map(0.5))))

2×2 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

### Measurement
Not implemented (yet).